## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense, Input
#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [4]:
# Create a list of at least 10 column names to use as X data
x_columns = [
    "Education",
    "Age",
    "DistanceFromHome",
    "JobSatisfaction",
    "OverTime",
    "StockOptionLevel",
    "WorkLifeBalance",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "NumCompaniesWorked",
]


# Create X_df using your selected columns
X_df = attrition_df[x_columns]

# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_df,
    y_df,
    test_size=0.2,
    random_state=1
)

In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
overtime_label_encoder = LabelEncoder()
overtime_label_encoder.fit(X_train['OverTime'])

X_train['OverTime'] = overtime_label_encoder.transform(X_train['OverTime'])
X_test['OverTime'] = overtime_label_encoder.transform(X_test['OverTime'])

X_train['OverTime'].value_counts()

OverTime
0    855
1    321
Name: count, dtype: int64

In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])

In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit_transform(y_train[["Attrition"]])

# Create two new variables by applying the encoder to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])


In [10]:
print(y_train_department_encoded.shape)
print(y_train_attrition_encoded.shape)


(1176, 3)
(1176, 2)


## Create, Compile, and Train the Model

In [11]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name= "input")

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation="relu", name= "shared_layer_1")(input_layer)
shared_layer_2 = layers.Dense(128, activation="relu", name= "shared_layer_2")(shared_layer_1)

In [12]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(32, activation="relu", name= "department_hidden_layer")(shared_layer_2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department_encoded.shape[1], activation="softmax", name= "department_output")(department_hidden_layer)


In [13]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = Dense(32, activation="relu", name= "attrition_hidden")(shared_layer_2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation= "sigmoid", name= "attrition_output")(attrition_hidden_layer)


In [14]:
# Create the model

model = Model(
    inputs = input_layer,
    outputs = [department_output_layer, attrition_output_layer]
)

# Compile the model
model.compile(
    optimizer="adam",
    loss={
        "department_output": "categorical_crossentropy",
        "attrition_output": "binary_crossentropy",
    },
    metrics={
        "department_output": "accuracy",
        "attrition_output": "accuracy",
    }
)


# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_1      │ (None, 64)        │        704 │ input[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer_2      │ (None, 128)       │      8,320 │ shared_layer_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden_… │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared_layer_2[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
model.fit(X_train_scaled,  
          [y_train_department_encoded, y_train_attrition_encoded],
          validation_data=(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded]),
          epochs=100, 
          verbose=2
)

Epoch 1/100
37/37 - 1s - 20ms/step - attrition_output_accuracy: 0.8070 - attrition_output_loss: 0.4826 - department_output_accuracy: 0.6429 - department_output_loss: 0.5281 - loss: 1.0099 - val_attrition_output_accuracy: 0.8027 - val_attrition_output_loss: 0.4672 - val_department_output_accuracy: 0.6327 - val_department_output_loss: 0.4749 - val_loss: 0.9449
Epoch 2/100
37/37 - 0s - 1ms/step - attrition_output_accuracy: 0.8478 - attrition_output_loss: 0.3819 - department_output_accuracy: 0.6590 - department_output_loss: 0.4829 - loss: 0.8635 - val_attrition_output_accuracy: 0.8095 - val_attrition_output_loss: 0.4551 - val_department_output_accuracy: 0.6327 - val_department_output_loss: 0.4726 - val_loss: 0.9317
Epoch 3/100
37/37 - 0s - 1ms/step - attrition_output_accuracy: 0.8597 - attrition_output_loss: 0.3554 - department_output_accuracy: 0.6590 - department_output_loss: 0.4769 - loss: 0.8341 - val_attrition_output_accuracy: 0.8095 - val_attrition_output_loss: 0.4782 - val_department

In [18]:
# Evaluate the model with the testing data
evaluation = model.evaluate(X_test_scaled, [y_test_department_encoded, y_test_attrition_encoded])

evaluation

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.7815 - attrition_output_loss: 1.8969 - department_output_accuracy: 0.5299 - department_output_loss: 1.1569 - loss: 3.06335


[3.0690724849700928,
 1.1511200666427612,
 1.8771793842315674,
 0.7925170063972473,
 0.5306122303009033]

In [19]:
# Print the accuracy for both department and attrition
print('Department Accuracy: %.3f' % evaluation[4])
print('Attrition Accuracy: %.3f' % evaluation[3])

Department Accuracy: 0.531
Attrition Accuracy: 0.793


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 